In [4]:
!pip install beautifulsoup4
!pip install requests
!pip install spacy
!python -m spacy download en_core_web_sm


2023-10-08 15:09:10.975137: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-08 15:09:12.257846: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# 1) Webscraping the Wikipedia

## NLTK basic version (general passwords)

In [ ]:
import requests
from bs4 import BeautifulSoup
import spacy
import random
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('stopwords')


# Initialize spaCy and other tools
nlp = spacy.load("en_core_web_sm")
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Initialize spaCy and other tools
nlp = spacy.load("en_core_web_sm")
stop_words = set(nltk.corpus.stopwords.words('english'))

def get_profiles_from_urls(url_list):
    profiles = {}
    for url in url_list:
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        name = soup.select_one('.firstHeading').text
        paragraphs = soup.select('.mw-parser-output p')
        profile_text = ' '.join([p.text for p in paragraphs[:]])  # take all paragraphs
        profiles[name] = profile_text
    return profiles

def preprocess_text(text):
    doc = nlp(text)
    tokens = [token for token in doc if not token.is_punct and not token.is_space]
    tokens = [token.text.lower() for token in tokens if token.text.lower() not in stop_words]
    return ' '.join(tokens)

def extract_named_entities(text):
    doc = nlp(text)
    named_entities = {}
    for ent in doc.ents:
        named_entities[ent.label_] = named_entities.get(ent.label_, [])
        if ent.text.lower().strip():
            named_entities[ent.label_].append(ent.text.lower())
    return named_entities

def generate_passwords(named_entities, num_passwords=10):
    possible_elements = []
    for category, entities in named_entities.items():
        possible_elements.extend([e for e in entities if e.strip()])

    passwords = []
    for _ in range(num_passwords):
        if len(possible_elements) >= 3:
            elements_to_use = random.sample(possible_elements, 3)
            random.shuffle(elements_to_use)
            password = "".join(elements_to_use)
            if random.choice([True, False]):
                password += str(random.randint(0, 99))
            passwords.append(password)
        else:
            passwords.append("Insufficient data")

    return passwords



In [ ]:

# List of Wikipedia URLs for notable people who died before 1980
urls = [
    'https://en.wikipedia.org/wiki/Albert_Einstein',
    'https://en.wikipedia.org/wiki/Isaac_Newton',
    'https://en.wikipedia.org/wiki/Galileo_Galilei',
    # ... add more URLs
]

# Step 1: Scrape Wikipedia
profiles = get_profiles_from_urls(urls)

# Step 2: Preprocess and Generate Passwords
for name, profile_text in profiles.items():
    print(f"Generating possible passwords for {name}")
    preprocessed_text = preprocess_text(profile_text)
    named_entities = extract_named_entities(preprocessed_text)
    passwords = generate_passwords(named_entities)
    print("Generated Passwords:")
    for i, password in enumerate(passwords, 1):
        print(f"{i}. {password}")
    print("\n")


Generating possible passwords for Albert Einstein
Generated Passwords:
1. 5][229united statesmach
2. 27institute agriculture chemical institute institute microbiologybrazil66
3. oskar halecki giuseppegerman1935
4. seventhroosevelt recommendingfive
5. 1908one191674
6. brownian1905claim.[255
7. 1922 1932hans albertfirst91
8. quantum194one
9. ethel michanowski berlinmax planck nielsone38
10. albert hallnew york 's1937 edition


Generating possible passwords for Isaac Newton
Generated Passwords:
1. 1999theologiancharles ii
2. first1988isaac barrow49
3. thirty yearsma.[22 time169920
4. 10,000 years agoorator university cambridgeknight strength
5. newtongodoxford university museum natural
6. londonsixteen years144 june 2020
7. 2001.[156john locke1665–166688
8. 149newtonorator university cambridge36
9. august 1665newton170143
10. mathematician nicolas fatio de duillier 16911701woolsthorpe colsterworth


Generating possible passwords for Galileo Galilei
Generated Passwords:
1. 1737twoscripture

## NLTK - easy & medium & difficult password generation

In [19]:
import requests
from bs4 import BeautifulSoup
import spacy
import random
import nltk
import string
import re


# Initialize spaCy and other tools
nlp = spacy.load("en_core_web_sm")
stop_words = set(nltk.corpus.stopwords.words('english'))

def get_profiles_from_urls(url_list):
    profiles = {}
    for url in url_list:
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        name = soup.select_one('.firstHeading').text
        paragraphs = soup.select('.mw-parser-output p')
        profile_text = ' '.join([p.text for p in paragraphs[:5]])  # Taking the first 5 paragraphs
        profiles[name] = profile_text
    return profiles

def preprocess_text(text):
    doc = nlp(text)
    tokens = [token for token in doc if not token.is_punct and not token.is_space]
    tokens = [token.text.lower() for token in tokens if token.text.lower() not in stop_words]
    return ' '.join(tokens)

def extract_named_entities(text):
    doc = nlp(text)
    named_entities = {}
    for ent in doc.ents:
        named_entities[ent.label_] = named_entities.get(ent.label_, [])
        if ent.text.lower().strip():
            named_entities[ent.label_].append(ent.text.lower())
    return named_entities


def clean_text(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)



def generate_passwords(named_entities, difficulty='easy', num_passwords=10):
    special_chars = string.punctuation
    possible_elements = []
    for category, entities in named_entities.items():
        possible_elements.extend([e for e in entities if e.strip()])

    passwords = []
    for _ in range(num_passwords):
        if len(possible_elements) >= 3:
            elements_to_use = random.sample(possible_elements, 3)
            random.shuffle(elements_to_use)
            password = "".join(elements_to_use)

            if difficulty == 'medium':
                password = password.capitalize()  # At least one uppercase
                password += str(random.randint(0, 99))  # At least one digit

            elif difficulty == 'difficult':
                password = password.capitalize()  # At least one uppercase
                password += random.choice(special_chars)  # At least one special character
                password += str(random.randint(0, 99))  # At least one digit

            passwords.append(password)
        else:
            passwords.append("Insufficient data")

    return passwords

# List of Wikipedia URLs for notable people who died before 1980
urls = [
    'https://en.wikipedia.org/wiki/Albert_Einstein',
    'https://en.wikipedia.org/wiki/Isaac_Newton',
    'https://en.wikipedia.org/wiki/Galileo_Galilei',
    # ... add more URLs
]



# Step 1: Scrape Wikipedia
profiles = get_profiles_from_urls(urls)


# Step 2: Preprocess and Generate Passwords
for name, profile_text in profiles.items():
    print("----------------------------------------")
    print(f"Generating possible passwords for {name}")
    cleaned_text = clean_text(profile_text)  # Clean the text first
    preprocessed_text = preprocess_text(cleaned_text)  # Then preprocess
    named_entities = extract_named_entities(preprocessed_text)
    easy_passwords = generate_passwords(named_entities, difficulty='easy')
    medium_passwords = generate_passwords(named_entities, difficulty='medium')
    difficult_passwords = generate_passwords(named_entities, difficulty='difficult')

    print("Easy Passwords:")
    for i, password in enumerate(easy_passwords, 1):
        print(f"{i}. {password}")

    print("\nMedium Passwords:")
    for i, password in enumerate(medium_passwords, 1):
        print(f"{i}. {password}")

    print("\nDifficult Passwords:")
    for i, password in enumerate(difficult_passwords, 1):
        print(f"{i}. {password}")

    print("\n")


----------------------------------------
Generating possible passwords for Albert Einstein
Easy Passwords:
1. united statesphd dissertation university zurichmarch april
2. germanyear ageyear
3. kaiser wilhelmgermanfirstly
4. secondlyhumboldt universityyear
5. theory relativityanazi warantan
6. fourjewshumboldt university
7. fourgermanbritish journal physics
8. onefirst decades twentieth centurysecondly
9. berlinbritish journal physicskaiser wilhelm
10. fourberlinadolf hitler

Medium Passwords:
1. Humboldt universityswissantan80
2. Theory relativityakaiser wilhelmgerman91
3. Fourkaiser wilhelmtheory relativitya48
4. World war iiberlingerman13
5. Yeargermananstan64
6. Twoyear ageswiss10
7. Phd dissertation university zurichantanamerican27
8. Switzerlandworld war iiswiss9
9. Swissunited statesfirstly9
10. United statesantanalbert einstein86

Difficult Passwords:
1. Germanyanstantheory relativitya$16
2. Kaiser wilhelmtheory relativityagerman*87
3. Adolf hitlergermanfour{5
4. Berlingermanam

## Distilbert - easy & medium & difficult

In [20]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.2 MB/s eta 0:00:00


In [23]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch

# Initialize DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def get_entity_embedding(entity, tokenizer, model):
    inputs = tokenizer(entity, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

def get_most_relevant_entities(named_entities, tokenizer, model):
    main_subject_embedding = get_entity_embedding("Albert Einstein", tokenizer, model)
    relevant_entities = []
    for category, entities in named_entities.items():
        for entity in entities:
            entity_embedding = get_entity_embedding(entity, tokenizer, model)
            similarity = torch.cosine_similarity(main_subject_embedding, entity_embedding)
            relevant_entities.append((entity, similarity))
    relevant_entities = sorted(relevant_entities, key=lambda x: x[1], reverse=True)[:10]
    return [entity for entity, similarity in relevant_entities]

# ... (rest of the code for scraping and preprocessing)
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove numbers and special characters
    text = re.sub(r'\[\d+\]', '', text)  # Remove citations like [1], [2], etc.
    return text

def get_profiles_from_urls(url_list):
    profiles = {}
    for url in url_list:
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        name = soup.select_one('.firstHeading').text
        paragraphs = soup.select('.mw-parser-output p')
        profile_text = ' '.join([p.text for p in paragraphs[:]])  # Taking the first 10 paragraphs for more context
        profiles[name] = profile_text
    return profiles


def preprocess_text(text):
    doc = nlp(text)
    tokens = [token for token in doc if not token.is_punct and not token.is_space]
    tokens = [token.text.lower() for token in tokens if token.text.lower() not in stop_words]
    return ' '.join(tokens)

def extract_named_entities(text):
    doc = nlp(text)
    named_entities = {}
    for ent in doc.ents:
        named_entities[ent.label_] = named_entities.get(ent.label_, [])
        if ent.text.lower().strip():
            named_entities[ent.label_].append(ent.text.lower())
    return named_entities




# Step 2: Preprocess, Get Relevant Entities, and Generate Passwords
for name, profile_text in profiles.items():
    print("----------------------------------------")
    print(f"Generating possible passwords for {name}")
    cleaned_text = clean_text(profile_text)
    preprocessed_text = preprocess_text(cleaned_text)
    named_entities = extract_named_entities(preprocessed_text)

    # Deduplication and sorting of entities
    for key in named_entities:
        named_entities[key] = list(set(named_entities[key]))

    relevant_entities = get_most_relevant_entities(named_entities, tokenizer, model)

    easy_passwords = generate_passwords({"PERSON": relevant_entities}, difficulty='easy')
    medium_passwords = generate_passwords({"PERSON": relevant_entities}, difficulty='medium')
    difficult_passwords = generate_passwords({"PERSON": relevant_entities}, difficulty='difficult')

    print("Easy Passwords:")
    for i, password in enumerate(easy_passwords, 1):
        print(f"{i}. {password}")

    print("\nMedium Passwords:")
    for i, password in enumerate(medium_passwords, 1):
        print(f"{i}. {password}")

    print("\nDifficult Passwords:")
    for i, password in enumerate(difficult_passwords, 1):
        print(f"{i}. {password}")

    print("\n")


----------------------------------------
Generating possible passwords for Albert Einstein
Easy Passwords:
1. british journal physicsprussian academy sciencesadolf hitler
2. adolf hitlerbritish journal physicsfranklin roosevelt
3. theory relativityajewsberlin
4. theory relativityajewsfranklin roosevelt
5. theory relativityahumboldt universitybritish journal physics
6. franklin rooseveltberlinalbert einstein
7. kaiser wilhelmalbert einsteinbritish journal physics
8. jewstheory relativityahumboldt university
9. humboldt universitytheory relativityaadolf hitler
10. albert einsteinjewskaiser wilhelm

Medium Passwords:
1. Kaiser wilhelmtheory relativityajews57
2. Albert einsteinberlintheory relativitya2
3. British journal physicsberlintheory relativitya64
4. Humboldt universityfranklin rooseveltadolf hitler95
5. British journal physicskaiser wilhelmprussian academy sciences45
6. Albert einsteinfranklin roosevelttheory relativitya62
7. British journal physicskaiser wilhelmadolf hitler97
8. K

## refined version for Distilbert

In [24]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch

# Initialize DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def get_entity_embedding(entity, tokenizer, model):
    inputs = tokenizer(entity, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

def get_most_relevant_entities(named_entities, tokenizer, model):
    main_subject_embedding = get_entity_embedding("Albert Einstein", tokenizer, model)
    relevant_entities = []
    for category, entities in named_entities.items():
        for entity in entities:
            entity_embedding = get_entity_embedding(entity, tokenizer, model)
            similarity = torch.cosine_similarity(main_subject_embedding, entity_embedding)
            relevant_entities.append((entity, similarity))
    relevant_entities = sorted(relevant_entities, key=lambda x: x[1], reverse=True)[:10]
    return [entity for entity, similarity in relevant_entities]




def fetch_profile(url):
    try:
        r = requests.get(url)
        r.raise_for_status()
        soup = BeautifulSoup(r.content, 'html.parser')
        return soup
    except requests.RequestException as e:
        print(f"An error occurred while fetching {url}: {e}")
        return None

def extract_name_and_text(soup):
    name = soup.select_one('.firstHeading').text
    paragraphs = soup.select('.mw-parser-output p')
    profile_text = ' '.join([p.text for p in paragraphs[:10]])  # Extracting first 10 paragraphs
    return name, profile_text

def validate_password(password, difficulty):
    if difficulty == 'medium':
        return any(char.isupper() for char in password) and any(char.isdigit() for char in password)
    elif difficulty == 'difficult':
        return (any(char.isupper() for char in password) and
                any(char.isdigit() for char in password) and
                any(char in string.punctuation for char in password))
    else:
        return True

# Main Execution
if __name__ == "__main__":
    urls = [
      'https://en.wikipedia.org/wiki/Albert_Einstein',
      'https://en.wikipedia.org/wiki/Isaac_Newton',
      'https://en.wikipedia.org/wiki/Galileo_Galilei',
      # ... add more URLs
  ]

    profiles = {}
    for url in urls:
        soup = fetch_profile(url)
        if soup:
            name, profile_text = extract_name_and_text(soup)
            profiles[name] = clean_text(profile_text)

    for name, profile_text in profiles.items():
        print("----------------------------------------")
        print(f"Generating possible passwords for {name}")

        preprocessed_text = preprocess_text(profile_text)
        named_entities = extract_named_entities(preprocessed_text)

        # Deduplicate entities
        for key in named_entities:
            named_entities[key] = list(set(named_entities[key]))

        relevant_entities = get_most_relevant_entities(named_entities, tokenizer, model)

        for difficulty in ['easy', 'medium', 'difficult']:
            passwords = generate_passwords({"PERSON": relevant_entities}, difficulty=difficulty)
            valid_passwords = [pw for pw in passwords if validate_password(pw, difficulty)]

            print(f"\n{difficulty.capitalize()} Passwords:")
            for i, password in enumerate(valid_passwords, 1):
                print(f"{i}. {password}")

        print("\n")

----------------------------------------
Generating possible passwords for Albert Einstein

Easy Passwords:
1. jewsfranklin roosevelthumboldt university
2. berlinfranklin roosevelthumboldt university
3. franklin rooseveltkaiser wilhelmalbert
4. franklin rooseveltkaiser wilhelmalbert
5. berlinalbert einsteinhumboldt university
6. jewsalbert einsteinbritish journal physics
7. jewsfranklin roosevelthumboldt university
8. jewsbritish journal physicstheory relativitya
9. adolf hitlerkaiser wilhelmfranklin roosevelt
10. albertadolf hitlerberlin

Medium Passwords:
1. Franklin roosevelttheory relativityabritish journal physics58
2. Franklin rooseveltadolf hitleralbert einstein17
3. Franklin roosevelttheory relativityahumboldt university36
4. Humboldt universitytheory relativityaadolf hitler26
5. Berlinfranklin rooseveltbritish journal physics95
6. Theory relativityaberlinhumboldt university49
7. Albertfranklin rooseveltbritish journal physics1
8. Jewskaiser wilhelmfranklin roosevelt17
9. Frank

# 2) Using a given text

In [25]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import spacy
import string
import random
import re

# Initialize NLP and DistilBERT models
nlp = spacy.load("en_core_web_sm")
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def clean_text(text):
    """Clean the text by removing special characters and citations."""
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\[\d+\]', '', text)
    return text

def preprocess_text(text):
    """Preprocess the text using spaCy to remove stopwords and punctuation."""
    doc = nlp(text)
    tokens = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(tokens)

def extract_named_entities(text):
    """Extract named entities using spaCy."""
    doc = nlp(text)
    named_entities = {}
    for ent in doc.ents:
        named_entities[ent.label_] = named_entities.get(ent.label_, [])
        if ent.text.lower().strip():
            named_entities[ent.label_].append(ent.text.lower())
    return named_entities

def get_entity_embedding(entity):
    """Get the DistilBERT embedding for a named entity."""
    inputs = tokenizer(entity, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

def get_most_relevant_entities(named_entities):
    """Get the most relevant entities based on DistilBERT embeddings."""
    relevant_entities = []
    for category, entities in named_entities.items():
        for entity in set(entities):  # Deduplicate entities
            entity_embedding = get_entity_embedding(entity)
            relevant_entities.append(entity)
    return relevant_entities[:10]  # Limit to 10 most relevant entities

def generate_passwords(relevant_entities, difficulty='easy', num_passwords=10):
    """Generate passwords based on relevant entities and difficulty level."""
    passwords = []
    for _ in range(num_passwords):
        elements = random.sample(relevant_entities, min(3, len(relevant_entities)))
        random.shuffle(elements)
        password = "".join(elements)

        if difficulty == 'medium':
            password = password.capitalize()
            password += str(random.randint(0, 99))
        elif difficulty == 'difficult':
            password = password.capitalize()
            password += random.choice(string.punctuation)
            password += str(random.randint(0, 99))

        passwords.append(password)
    return passwords

# Main Execution
if __name__ == "__main__":
    profile_text = """
    Galileo Galilei was an Italian astronomer, physicist and engineer, sometimes described as a polymath.
    He has been called the 'father of observational astronomy',
    the 'father of modern physics', and the 'father of the scientific method'.
    """

    print(f"Generating possible passwords for Galileo Galilei")

    cleaned_text = clean_text(profile_text)
    preprocessed_text = preprocess_text(cleaned_text)
    named_entities = extract_named_entities(preprocessed_text)
    relevant_entities = get_most_relevant_entities(named_entities)

    for difficulty in ['easy', 'medium', 'difficult']:
        passwords = generate_passwords(relevant_entities, difficulty)
        print(f"\n{difficulty.capitalize()} Passwords:")
        for i, password in enumerate(passwords, 1):
            print(f"{i}. {password}")


Generating possible passwords for Galileo Galilei

Easy Passwords:
1. italian
2. italian
3. italian
4. italian
5. italian
6. italian
7. italian
8. italian
9. italian
10. italian

Medium Passwords:
1. Italian83
2. Italian78
3. Italian43
4. Italian8
5. Italian31
6. Italian43
7. Italian58
8. Italian68
9. Italian29
10. Italian72

Difficult Passwords:
1. Italian,53
2. Italian^11
3. Italian'23
4. Italian@2
5. Italian>75
6. Italian/33
7. Italian&35
8. Italian+15
9. Italian!90
10. Italian&62
